In [1]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [2]:
# 获取训练数据，测试数据
train_data = []
with open('./after_preprocess_traindata.txt',encoding='utf-8') as f:
    for line in f:
        train_data.append(line.strip())

test_data = []
with open('./after_preprocess_testdata.txt',encoding='utf-8') as f:
    for line in f:
        test_data.append(line.strip())

In [3]:
# 获取标签编码
df1 = pd.read_csv('../dataset/cnews/cnews.train.txt',sep='\t',names=['label','content'],encoding='UTF-8',engine='python')
df2 = pd.read_csv('../dataset/cnews/cnews.test.txt',sep='\t',names=['label','content'],encoding='UTF-8',engine='python')
encoder = LabelEncoder()

train_y = encoder.fit_transform(df1['label'])
test_y = encoder.transform(df2['label'])

### 接着我们要把词转化为词频向量（词袋），注意由于LDA是基于词频统计的，因此一般不用TF-IDF来做文档特征。

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
bow = cv.fit_transform(train_data)
# print(bow)

In [7]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(mutual_info_classif, k=5000)
new_train_x = selector.fit_transform(bow,train_y)
print(new_train_x.shape)

--4--
(50000, 5000)


#### 保存剩余筛选器模型

In [12]:
import pickle
pkl = pickle.dumps(selector)
with open('./selector_model.pkl','wb') as f:
    f.write(pkl)

#### 测试读取筛选器

In [14]:
with open('./selector_model.pkl','rb') as f:
    model = pickle.loads(f.read())
new_train_x_test = model.transform(bow)
print(new_train_x_test.shape)

(50000, 5000)


### 开始创建LDA模型

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=150, learning_offset=50.,max_iter=1000,random_state=1)
lda.fit(new_train_x)